In [1]:
import MetaTrader5 as mt5
from datetime import datetime
import numpy as np
import pandas as pd
import time
mt5.initialize()

True

In [6]:
mt5.positions_get()

(TradePosition(ticket=71287076, time=1678210485, time_msc=1678210485858, time_update=1678210485, time_update_msc=1678210485858, type=0, magic=0, identifier=71287076, reason=0, volume=0.01, price_open=1.06122, sl=1.05633, tp=0.0, price_current=1.05931, swap=0.0, profit=-2.89, symbol='EURUSD.a', comment='', external_id=''),
 TradePosition(ticket=71288057, time=1678211108, time_msc=1678211108510, time_update=1678211108, time_update_msc=1678211108510, type=0, magic=0, identifier=71288057, reason=0, volume=0.01, price_open=1.0596, sl=1.05401, tp=0.0, price_current=1.05931, swap=0.0, profit=-0.44, symbol='EURUSD.a', comment='', external_id=''),
 TradePosition(ticket=71289575, time=1678211636, time_msc=1678211636702, time_update=1678211636, time_update_msc=1678211636702, type=0, magic=0, identifier=71289575, reason=0, volume=0.01, price_open=1.05839, sl=1.05185, tp=0.0, price_current=1.05931, swap=0.0, profit=1.3900000000000001, symbol='EURUSD.a', comment='', external_id=''))

In [7]:
def find_filling_mode(symbol):
        for i in range(2):
            request = {
            "action": mt5.TRADE_ACTION_DEAL,
            "symbol": symbol,
            "volume": mt5.symbol_info(symbol).volume_min,
            "type": mt5.ORDER_TYPE_BUY,
            "price": mt5.symbol_info(symbol).ask,
            "type_filling": i,
            "type_time": mt5.ORDER_TIME_GTC}
            result = mt5.order_check(request)
            if result.comment == "Done":
                break
        return i

In [8]:
def resume():

    columns = ["ticket", "position", "symbol", "volume", "magic", "profit", "price", "tp", "sl", "trade_size"]

    list = mt5.positions_get()

    summary = pd.DataFrame()

    for element in list:
        element_pandas = pd.DataFrame([element.ticket, element.type, element.symbol, element.volume, element.magic,
                                       element.profit, element.price_open, element.tp, element.sl,
                                       mt5.symbol_info(element.symbol).trade_contract_size],
                                       index=columns).transpose()
        
        summary = pd.concat((summary, element_pandas), axis=0)


    try:
        summary["profit %"] = summary.profit / (summary.price * summary.trade_size * summary.volume)
        summary = summary.reset_index(drop=True)
    except:
        pass
    return summary

In [5]:
resume()

,ticket,position,symbol,volume,magic,profit,price,tp,sl,trade_size,profit %
0,71287076,0,EURUSD.a,0.01,0,-2.83,1.06122,0.0,1.05633,100000.0,-0.002667
1,71288057,0,EURUSD.a,0.01,0,-0.38,1.0596,0.0,1.05401,100000.0,-0.000359
2,71289575,0,EURUSD.a,0.01,0,1.45,1.05839,0.0,1.05185,100000.0,0.00137


In [6]:
mt5.symbol_info('EURUSD.a')

SymbolInfo(custom=False, chart_mode=0, select=True, visible=True, session_deals=0, session_buy_orders=0, session_sell_orders=0, volume=0, volumehigh=0, volumelow=0, time=1678210583, digits=5, spread=10, spread_float=True, ticks_bookdepth=0, trade_calc_mode=5, trade_mode=4, start_time=0, expiration_time=0, trade_stops_level=0, trade_freeze_level=0, trade_exemode=2, swap_mode=1, swap_rollover3days=3, margin_hedged_use_leg=False, expiration_mode=15, filling_mode=2, order_mode=127, order_gtc_mode=0, option_mode=0, option_right=0, bid=1.06136, bidhigh=1.06939, bidlow=1.05855, ask=1.06146, askhigh=1.06951, asklow=1.05868, last=0.0, lasthigh=0.0, lastlow=0.0, volume_real=0.0, volumehigh_real=0.0, volumelow_real=0.0, option_strike=0.0, point=1e-05, trade_tick_value=1.508409382306358, trade_tick_value_profit=1.508409382306358, trade_tick_value_loss=1.5087052291723244, trade_tick_size=1e-05, trade_contract_size=100000.0, trade_accrued_interest=0.0, trade_face_value=0.0, trade_liquidity_rate=0.0,

**TRAILING STOP LOSS**

In [9]:
# Create a dictionary to contain the maximum price of each symbol.
max_price = dict()
min_price = dict()

# Infinite loop
while True:

    summary = resume()

    # Verification: Is there any open position?
    if summary.shape[0] > 0:
        for i in range(summary.shape[0]):

            # Extract information
            row = summary.iloc[i]
            symbol = row["symbol"]

            # Add the key if it is not in the keys list
            if symbol not in max_price.keys():
                max_price[symbol]=row["price"]

            if symbol not in min_price.keys():
                min_price[symbol]=row["price"]

            """ CASE 1: Change dynamically the stop loss for a BUY ORDER """
            # Trailing stop loss for a buy order, 0 = buy, 1 = sell
            if row["position"] == 0:
                # print(row)
                # Extract current price
                current_price = (mt5.symbol_info(symbol).ask + mt5.symbol_info(symbol).bid) / 2

                # Compute distance between current price and max price
                from_sl_to_current_price = current_price - row["sl"]
                from_sl_to_max_price = max_price[symbol] - row["sl"]

                # If current price is greater than previous max price --> new max price
                if current_price > max_price[symbol]:
                    max_price[symbol] = current_price

                # Find the difference between the current minus max
                if from_sl_to_current_price > from_sl_to_max_price:
                    difference = from_sl_to_current_price - from_sl_to_max_price

                    filling_type = find_filling_mode(symbol)

                    point = mt5.symbol_info(symbol).point

                    request = {
                        "action": mt5.TRADE_ACTION_SLTP,
                        "symbol": symbol,
                        "position": row["ticket"],
                        "volume": row["volume"],
                        "type": mt5.ORDER_TYPE_BUY,
                        "price": row["price"],
                        "sl": row["sl"] + difference,
                        "type_filling": filling_type,
                        "type_time": mt5.ORDER_TIME_GTC
                    }

                    information = mt5.order_send(request)
                    # print(information)
                    

